In [9]:
import pandas as pd

In [10]:
is_kaggle = False

if is_kaggle:
    base_dir = '/kaggle/input'
    data_dir = f'{base_dir}/linking-writing-processes-to-writing-quality'
    output_dir = '/kaggle/working'
else:
    base_dir = '../'
    data_dir = f'{base_dir}/0_data'
    models_dir = f'{base_dir}/models'

In [11]:
# train_logs=pd.read_csv(data_dir+"/train_logs.csv")

In [14]:
train_logs=pd.read_csv(data_dir+"/train_logs.csv")
print(f"len(train_logs):{len(train_logs)}")
train_logs=train_logs.sort_values(by=['id', 'down_time'])
train_logs = train_logs.reset_index(drop=True)
train_logs['event_id'] = train_logs.groupby('id').cumcount() + 1
# train_logs.to_csv("train_logs_corrected.csv",index=None)

train_scores=pd.read_csv(data_dir+"/train_scores.csv")

test_logs=pd.read_csv(data_dir+"/test_logs.csv")
print(f"len(test_logs):{len(test_logs)}")
test_logs=test_logs.sort_values(by=['id', 'down_time'])
test_logs = test_logs.reset_index(drop=True)
test_logs['event_id'] = test_logs.groupby('id').cumcount() + 1
# test_logs.to_csv("test_logs_corrected.csv",index=None)

len(train_logs):8405898
len(test_logs):6


In [15]:
def process_group(group):
    # Find the first row where 'activity' equals 'Input' and 'text_change' equals 'q'
    first_input_q_index = group[(group['activity'] == 'Input') & (group['text_change'] == 'q')].index[0]

    # Calculate the 'down_time' of the first 'Input'/'q' row
    idle_time = group.loc[first_input_q_index, 'down_time']

    # Add it to the 'idle_time' column
    group['idle_time'] = idle_time

    # Drop all rows before this row
    group = group.loc[first_input_q_index:]

    # Subtract the 'down_time' of the first row from all 'down_time' values in the group
    group['up_time'] -= group['down_time'].iloc[0]
    group['down_time'] -= group['down_time'].iloc[0]

    return group

train_logs = train_logs.groupby('id').apply(process_group).reset_index(drop=True)
train_logs['event_id'] = train_logs.groupby('id').cumcount() + 1
train_logs.to_csv("train_logs_double_corrected.csv", index=None)

# test_logs = test_logs.groupby('id').apply(process_group).reset_index(drop=True)
# test_logs['event_id'] = test_logs.groupby('id').cumcount() + 1
# test_logs.to_csv("test_logs_double_corrected.csv", index=None)

In [16]:
first_10min = 10*60*1000
first_20min = 20*60*1000

df_first = train_logs[train_logs['up_time'] < first_10min]
df_first.to_csv('./train_logs_double_corrected_first_10min.csv', index=False)

df_second = train_logs[(train_logs['up_time'] >= first_10min)]# & (train_logs['up_time'] < first_20min)]
# df_third = train_logs[train_logs['up_time'] >= first_20min]

df_second.to_csv('./train_logs_double_corrected_second_10min.csv', index=False)
# df_third.to_csv('.train_logs_double_corrected_third_10min.csv', index=False)